In [1]:
#library importation
import csv
import numpy as np
import random
import torch
import torch.utils.data
import pandas as pd


import time
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import seaborn as sn  # for heatmaps
from sklearn.metrics import confusion_matrix
from torchvision.utils import make_grid



import matplotlib.pyplot as plt # for plotting
import torch.optim as optim #for gradient descent

import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import shutil
from torchvision.datasets import ImageFolder

import ast

# ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Getting URL

In [2]:
#public url
id = '1T1zNm7WxRL43_szet_ZfRN2AwrE1U5tF'
url = 'https://drive.google.com/uc?id={}'.format(id)
df = pd.read_csv(url)

print(df)

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [ ]:
# winnie's personal path upload
path = '/content/mfcc_data.csv'
df = pd.read_csv(url)

In [4]:
path1 = "/kaggle/input/mfcc-data/mfcc_data.csv"

df1 = pd.read_csv(path1)

categories = df1.iloc[:, 1]
print(categories)

0      blues
1      blues
2      blues
3      blues
4      blues
       ...  
994     rock
995     rock
996     rock
997     rock
998     rock
Name: label, Length: 999, dtype: object


In [5]:
path2 = "/kaggle/input/mfcc-fma/mfcc_fma.csv"

df2 = pd.read_csv(path2)

categories = df2.iloc[:, 1]
print(categories)

0        hiphop
1        hiphop
2           pop
3       country
4       country
         ...   
3919     hiphop
3920     hiphop
3921        pop
3922        pop
3923     hiphop
Name: label, Length: 3924, dtype: object


In [6]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class MFCCDataset(Dataset):
    def __init__(self, df, transform = None):
        self.df = df
        self.mfccs = self.df.iloc[:, 0]
        self.categories = encoding(self.df.iloc[:, 1])
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        mfcc = convert2NumpyArray(self.mfccs[idx])
        category = self.categories[idx]
        if self.transform is not None:
            mfcc = self.transform(mfcc)

        return mfcc, category

genres = ['pop', 'metal', 'disco', 'blues', 'reggae', 'classical', 'rock', 'hiphop', 'country', 'jazz']
def encoding(categories):
  update_category = []
  for i in range(len(categories)):
    if categories[i] in genres:
      update_category.append(genres.index(categories[i]))
    else:
      print("error")
  return update_category


def convert2NumpyArray(mfccs):
    row_num = -1
    for ch in mfccs:
        if ch == '[':
            row_num += 1
    # Remove unnecessary characters
    cleaned_input = mfccs.replace("[", "").replace("]", "").replace("\n", "")

    # Split the string into individual elements
    elements = cleaned_input.split()

    # Calculate the number of columns and rows
    column_num = len(elements) // row_num

    # Reshape the array to the desired shape
    numeric_array = np.array(elements, dtype=np.float64)
    numeric_array = numeric_array.astype(np.float32)  # Cast to float32
    #desired_shape = (row_num, column_num)
    reshaped_array = numeric_array.reshape(numeric_array.shape + (1,))
    return reshaped_array

output_shape = (20, 1000)

# Create the transform
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(output_shape),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = MFCCDataset(df1, transform = transform)
dataset2 = MFCCDataset(df2, transform = transform)
dataset = torch.utils.data.ConcatDataset([dataset, dataset2])

In [7]:
def augment(df, target_size, save=False):

  '''takes input of pandas dataframe and augments it to desired size, returns dataframe containing new data'''
  from random import randint, choice

  labels = list(range(target_size))
  datas = list(range(target_size))
  lengths = [5, 10, 15, 20, 25]


  for i in range(target_size):
    data, label = df.loc[randint(0,len(df)-1)]
    data = convert2NumpyArray(data)
    mask = randint(0,1)

    if mask:
      mask_len = int(1000/30 * choice(lengths))
      zeros = np.zeros((mask_len, 1))
      begin = randint(0, 1000 - mask_len)
      data[begin:begin+mask_len, :] = zeros

    else:
      keep_len = int(1000/30 * choice(lengths))
      begin = randint(0, 1000 - keep_len)
      data[begin, :] = 0
      data[ begin+keep_len:, :] = 0

    datas[i] = np.array2string(data,threshold=np.inf)
    labels[i] = label

  df = ({
    'mfcc':datas,
    'labels':labels
               })
  df = pd.DataFrame(df)

  if save:
    df.to_csv(f'mfcc_data_{target_size}.csv', index=False)

  return df



In [ ]:
aug_df1 = augment(df1, 1000, save=True)
aug_dataset = MFCCDataset(aug_df1, transform = transform)

dataset = torch.utils.data.ConcatDataset([dataset, aug_dataset])

In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as data
def get_data_loader(dataset, batch_size=1,data_size = -1):

    split_sizes = [0.6,0.2,0.2] #60% of training data, 20% of validation data, and 20% of testing data
    train_dataset,val_dataset,test_dataset = data.random_split(dataset, split_sizes , generator=torch.Generator().manual_seed(42))
    train_loader = data.DataLoader(train_dataset, batch_size=batch_size, num_workers=1, shuffle=True)
    val_loader = data.DataLoader(val_dataset, batch_size=batch_size, num_workers=1, shuffle=False)
    test_loader = data.DataLoader(test_dataset, batch_size=batch_size,  num_workers=1, shuffle=False)
    return train_loader, val_loader, test_loader

In [ ]:
train_loader, val_loader, test_loader = get_data_loader(dataset)
print(train_loader, len(train_loader)) #bcz batch size is 10

In [ ]:
i = 0
for img, labels in train_loader:
    data = img  # Assuming each batch contains a single data instance
    #print(data.shape)
    #print(labels)
    i+= 1
    if i== 20:
      break

genres = ['pop', 'metal', 'disco', 'blues', 'reggae', 'classical', 'rock', 'hiphop', 'country', 'jazz']
np.set_printoptions(formatter=dict(int=lambda x: f'{x:5}')) # to widen the printed array

# Print the labels
print('Label:', labels.numpy())
print('Class: ', *np.array([genres[i] for i in labels]))

# Print the images
im = make_grid(img, nrow=1)  # the default nrow is 8
plt.figure(figsize=(10,4))
plt.imshow(np.transpose(im.numpy(), (1, 2, 0)));

In [ ]:
class CNN_RNN_Hybrid(nn.Module):
    def __init__(self):
        super(CNN_RNN_Hybrid, self).__init__()
        self.name = "music_genre_classification"

        # 2 conv layers + max pooling
        self.conv1 = nn.Conv2d(1, 6, 3, 1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3, 1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)


        # rnn (LSTMs: heard that this might be better)
        self.rnn = nn.LSTM(input_size=5*25*16, hidden_size=1024, num_layers=3, batch_first=True)
        # Apply dropout
        self.dropout = nn.Dropout(0.20)


        # fully connected layer
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 10)  # 10 is the number of genres

    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 5*25*16)

        out_rnn, _ = self.rnn(X.unsqueeze(0))  # Add unsqueeze to add batch dimension
        X_rnn = out_rnn[:, -1, :]  # Take the last hidden state of the RNN


        X = F.relu(self.fc1(X_rnn))
        #Apply Dropout
        X = self.dropout(X)
        X = F.relu(self.fc2(X))
        X = self.dropout(X)
        X = F.relu(self.fc3(X))
        X = self.fc4(X)
        X = X.squeeze(1)


        return X

In [ ]:
model = CNN_RNN_Hybrid()
print(model)


In [ ]:
#Winnie

#helper function get_model_name, get_accuracy, get_loss

def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values
    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path

#########################################################################
def get_accuracy(model, batch_size, train=False, test= False):
    if train:
        data = train_loader
    elif test:
        data = test_loader
    else:
      data = val_loader

    correct = 0
    total = 0
    model.eval()
    for spectrums, labels in data:

      #############################################
      # To Enable GPU Usage
      if use_cuda and torch.cuda.is_available():
        spectrums = spectrums.cuda()
        labels = labels.cuda()
      #############################################

      output = model(spectrums)

      #select index with maximum prediction score
      pred = output.max(1, keepdim=True)[1] # get the index of the max logit
      correct += pred.eq(labels.view_as(pred)).sum().item()
      total += spectrums.shape[0]
    return correct / total

def get_loss(model, data_loader, criterion):
  total_loss = 0
  i = 0
  for data, labels in data_loader:
    if use_cuda and torch.cuda.is_available():
          data = data.cuda()
          labels = labels.cuda()
    data = torch.tensor(data)
    out = model(data)
    loss = criterion(out, labels)
    total_loss += loss.item()
    i += 1

  return total_loss/(i + 1)

In [ ]:
#Winnie
learning_rate = 0.00001
weight_decay = 0.0001
batch_size = 500
#Training


def train(model, train_loader, valid_loader, batch_size, num_epochs, learning_rate=learning_rate, weight_decay=weight_decay):
    """ Training loop. You should update this."""
    torch.manual_seed(42)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),weight_decay=weight_decay,lr=learning_rate)

    train_loss = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    train_acc = np.zeros(num_epochs)
    val_acc = np.zeros(num_epochs)

    start_time = time.time()

    device = torch.device("cuda")
    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        n = 0 # the number of iterations
        for data, labels in train_loader:

            #############################################
            # To Enable GPU Usage
            if use_cuda and torch.cuda.is_available():
              data = data.cuda()
              labels = labels.cuda()
            #############################################
            #print("data: ", data.shape)

            out = model(data)             # forward pass

            #print("outs(this should be 16x10): ", out.shape)
            #print("Labels: " , labels)
            loss = criterion(out, labels) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch



            # Save the current training information

            total_loss += loss.item()
            n += 1

        train_acc[epoch] = get_accuracy(model, batch_size= batch_size, train=True)
        val_acc[epoch] = get_accuracy(model, batch_size= batch_size)
        train_loss[epoch] = total_loss/(n + 1)
        val_loss[epoch] = get_loss(model, valid_loader, criterion)


        print(("Epoch {}: Train acc: {} |"+"Validation acc: {}").format(
                epoch+1,
                train_acc[epoch],
                val_acc[epoch]))

        # Save the current model (checkpoint) to a file
        model_path = get_model_name(model.name, batch_size, learning_rate, epoch)
        torch.save(model.state_dict(), model_path)

    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.4f} seconds".format(elapsed_time))


    # plotting
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.title("Training Curve")
    plt.plot(range(1 ,num_epochs+1), train_loss, label="Train")
    plt.plot(range(1 ,num_epochs+1), val_loss, label="Validation")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")

    plt.subplot(1,2,2)
    plt.title("Training Curve")
    plt.plot(range(1 ,num_epochs+1), train_acc, label="Train")
    plt.plot(range(1 ,num_epochs+1), val_acc, label="Validation")
    plt.xlabel("Epochs")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))

    return train_acc, val_acc, train_loss, val_loss, num_epochs

In [ ]:
realmodel6 = CNN_RNN_Hybrid()
use_cuda = True
print("realmodel6")
realmodel6.name = "realmodel6"


train_acc, val_acc, train_loss, val_loss, epochs = train(
    realmodel6, train_loader, val_loader,
    batch_size=1, num_epochs=10, learning_rate=0.00001, weight_decay=0.0001
)